# Demo de GraphRAG  
Este notebook muestra cómo:
- Convertir descripciones de productos en vectores (“embeddings”).  
- Buscar productos por similitud semántica.  
- Contextualizar consultas usando metadatos del grafo.  
- Generar respuestas enriquecidas con LLM.  

## 0. Preparación Entorno
- Librerías Python
- Instancia Neo4j
- Key OpenAI

In [2]:
import os
from dotenv import load_dotenv
from neo4j import GraphDatabase, RoutingControl
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores.neo4j_vector import Neo4jVector
from langchain import PromptTemplate, LLMChain
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.chat_models import ChatOpenAI
from langchain.schema import Document

load_dotenv(override=True)
uri = os.getenv("NEO4J_URI")
user = os.getenv("NEO4J_USER")
password = os.getenv("NEO4J_PASSWORD")
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY')
embedding_model = OpenAIEmbeddings()

## 1. Carga de datos
Desde la consola de **Aura** a partir de un CSV

## 2. Embeddings y Similaridad

En esta sección conectamos con Neo4j, generamos los embeddings de los nodos `Product` y los extraemos para poder hacer búsquedas semánticas rápidas


In [ ]:
vectorstore = Neo4jVector.from_existing_graph(
    embedding=embedding_model,
    url=uri,
    username=user,
    password=password,
    index_name='product',
    node_label="Product",
    embedding_node_property="embedding",
    text_node_properties=["description","name"]
)

> Dada una frase de usuario, recuperamos los 𝐤 productos cuyas `descripciones` estén más cerca en el espacio semántico. Incluye score. 

In [3]:
#question = """Busco un frigorífico silencioso para un apartamento pequeño."""
question = """¿Qué horno es adecuado para familias numerosas?"""
#question = """Quiero un lavavajillas con control por voz."""
#question = """¿Qué lavadora está diseñada para uso frecuente y tiene diseño práctico?"""

In [ ]:
results = vectorstore.similarity_search_with_score(question, k=3)
for doc, score in results:
    print(f"Score: {score:.4f}")
    print(f"{doc.page_content}")
    print("---")

Score: 0.9167

description: HeatWave X631 es un horno sostenible y ideal para familias numerosas. Incorpora estructura interna modular, lo que lo hace destacar frente a otros modelos de su clase. Su diseño responde a necesidades específicas con un enfoque práctico y funcional.
name: KüchenPro HeatWave X631
---
Score: 0.9149

description: HeatMaster Q114 es un horno versátil y ideal para familias numerosas. Incorpora algoritmos de optimización energética, lo que lo hace destacar frente a otros modelos de su clase. Su diseño responde a necesidades específicas con un enfoque práctico y funcional.
name: HomeEase HeatMaster Q114
---
Score: 0.9106

description: SteamChef Z618 es un horno versátil y ideal para familias numerosas. Incorpora algoritmos de optimización energética, lo que lo hace destacar frente a otros modelos de su clase. Su diseño responde a necesidades específicas con un enfoque práctico y funcional.
name: ChillCore SteamChef Z618
---


## 3. Contextualización con Neo4j  
1. Tras la búsqueda semántica, usamos Cypher para obtener `categoría` y `marca`.
2. Combinamos vectores + grafo para **enriquecer la respuesta** con `rating`.

In [ ]:
enrich_for_summary_query = """
match (c:Category)<-[:BELONGS_TO]-(node:Product)-[:BRANDED_BY]->(b:Brand)
optional match (node)<-[r:RATING]-(:Customer)
with node, c, b, avg(r.value) as avg_rating, score
return node.description AS text, score,
    node {id: node.productid, name: node.name, description: node.description,  brand: b.name, category: c.name, avg_rating: avg_rating} 
as metadata
"""

contextualized_vectorstore = Neo4jVector.from_existing_index(
    OpenAIEmbeddings(),
    url=uri,
    username=user,
    password=password,
    index_name="product",
    retrieval_query=enrich_for_summary_query,
)

> Se ejecuta de nuevo similarity, esta vez con el contexto del grafo

In [ ]:
def kg_product_recommendations(input):
  response = contextualized_vectorstore.similarity_search_with_score(input, k=3)
  return "\n".join([
        f"- {doc.metadata.get('name')}: {doc.page_content} Rating: {doc.metadata.get('avg_rating')}. Categoria: {doc.metadata.get('category')}. Marca: {doc.metadata.get('brand')}."
        for doc, score in response
    ])

In [ ]:
print(kg_product_recommendations(question))

- KüchenPro HeatWave X631: HeatWave X631 es un horno sostenible y ideal para familias numerosas. Incorpora estructura interna modular, lo que lo hace destacar frente a otros modelos de su clase. Su diseño responde a necesidades específicas con un enfoque práctico y funcional. Rating: 2.2. Categoria: Horno. Marca: KüchenPro.
- HomeEase HeatMaster Q114: HeatMaster Q114 es un horno versátil y ideal para familias numerosas. Incorpora algoritmos de optimización energética, lo que lo hace destacar frente a otros modelos de su clase. Su diseño responde a necesidades específicas con un enfoque práctico y funcional. Rating: 3.0. Categoria: Horno. Marca: HomeEase.
- ChillCore SteamChef Z618: SteamChef Z618 es un horno versátil y ideal para familias numerosas. Incorpora algoritmos de optimización energética, lo que lo hace destacar frente a otros modelos de su clase. Su diseño responde a necesidades específicas con un enfoque práctico y funcional. Rating: 2.6666666666666665. Categoria: Horno. Mar

## 4. Generación de Mensaje con LLM  
Usamos un `Prompt Template` para:
- Listar los productos hallados (con marca y categoría).  
- Pedir al modelo que recomiende y explique ventajas.
- Usando lenguaje natural.

In [ ]:
template = '''
Eres un asistente de productos de electrodomésticos. El usuario ha preguntado:
  "{user_query}"

Estos son los 3 productos más relevantes encontrados (con descripción):
{top_products}

Genera una respuesta completa que:
 - Explique brevemente por qué estos productos son adecuados, indicando en la explicación de cada uno la marca, la categoría y destaque un punto fuerte.
 - Ordene por el avg_rating de los productos.
 - Recomiende uno de ellos, el de avg_rating más alto.
 - Invite al usuario a preguntar más detalles.
'''

prompt = PromptTemplate(
    input_variables=["user_query", "top_products"],
    template=template
)

> Inicializa el modelo GPT-4o y configura un flujo (LLMChain) que utiliza ese modelo junto al prompt predefinido.

In [ ]:
llm = ChatOpenAI(model="gpt-4o", temperature=0.7)
chain = LLMChain(llm=llm, prompt=prompt)

> Se obtienen las recomendaciones (top_products) y se genera el flujo junto con la consulta del usuario

In [ ]:
def humanized_recommendation(question, k = 3):
    top_products = kg_product_recommendations(question)
    enriched = chain.predict(user_query=question, top_products=top_products)
    return enriched

In [ ]:
#question = """Busco un frigorífico silencioso para un apartamento pequeño."""
question = """¿Qué horno es adecuado para familias numerosas?"""
#question = """Quiero un lavavajillas con control por voz."""
#question = """¿Qué lavadora está diseñada para uso frecuente y tiene diseño práctico?"""

In [ ]:
response = humanized_recommendation(question)
print(response)

¡Hola! Aquí tienes tres opciones de hornos que podrían ser adecuados para familias numerosas, ordenados según su calificación promedio:

1. **HomeEase HeatMaster Q114**
   - **Categoría:** Horno
   - **Marca:** HomeEase
   - **Punto fuerte:** Este modelo es versátil e incorpora algoritmos de optimización energética, lo que lo hace destacar en eficiencia. Es ideal para familias numerosas gracias a su diseño práctico y funcional.
   - **Calificación:** 3.0

2. **ChillCore SteamChef Z618**
   - **Categoría:** Horno
   - **Marca:** ChillCore
   - **Punto fuerte:** Este horno también es versátil y está diseñado para satisfacer las necesidades de familias grandes. Se destaca por su enfoque en la optimización energética, asegurando un uso eficiente de los recursos.
   - **Calificación:** 2.67

3. **KüchenPro HeatWave X631**
   - **Categoría:** Horno
   - **Marca:** KüchenPro
   - **Punto fuerte:** Destaca por su estructura interna modular, lo que le confiere una gran flexibilidad y lo hace id

graphRAG >> Facilidad y contexto